In [1]:
import numpy as np
import requests
import time

from pyspark import SQLContext, SparkContext
from pyspark import SparkConf

from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.linalg import Vectors

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold

from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

In [2]:
# Load the dataset and labels
x=np.load('RMCx.npy')
y=np.load('RMCy.npy')

In [3]:
np.unique(y).shape

In [4]:
conf = SparkConf()
sc = SparkContext(conf = conf)
spark = SQLContext(sc)

In [5]:
# Calculate the rank of each feature
R=[]
for h in range(x.shape[1]):
    kmeans = KMeans(init='k-means++', n_clusters=np.unique(y).shape[0])
    ff=kmeans.fit_predict(x[:,h].reshape(-1,1))
    r=metrics.homogeneity_score(y,ff) #Use the homogeneity score as a rank of the feature
    R.append(r)

In [6]:
#Initiate the cross-validation splitter
kfolds=StratifiedKFold(n_splits=5,shuffle=True)

In [7]:
#Per each set of ranks, use cross-validation to calculate accuracy.
smr=[]
et=0
for j in range(Rnk.shape[0]):
    fd=x[:,Rnk[j:]]
    pp=0
    lpa=np.zeros((0,2))
    for train,test in kfolds.split(fd,y):
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[train],y[train].reshape(-1,1))))
        TrD = spark.createDataFrame(dff,schema=["label", "features"]).rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))
        dff = map(lambda x: (int(float(x[-1])), Vectors.dense(x[:-1])),np.hstack((fd[test],y[test].reshape(-1,1))))
        TsD = spark.createDataFrame(dff,schema=["label", "features"]).rdd.map(lambda row: LabeledPoint(row.label, MLLibVectors.fromML(row.features)))
        model = RandomForest.trainClassifier(TrD, numClasses=np.unique(y).shape[0],categoricalFeaturesInfo={},numTrees=100) #The number of classes in the dataset
        predictions = model.predict(TsD.map(lambda x: x.features))
        st = time.time()
        labelsAndPredictions = TsD.map(lambda lp: lp.label).zip(predictions)
        lpa=np.vstack((lpa,labelsAndPredictions.toDF().toPandas()))
        et+=time.time()-st
        acc = labelsAndPredictions.filter(lambda lp: lp[0] == lp[1]).count() / float(TsD.count())
        pp=pp+acc
    pp=pp/kfolds.n_splits
    np.savetxt('F%d.csv'%j,lpa,delimiter=',')
    smr.append([j, pp, et*1000000/x.shape[0]]) #Calculate the time requires to predict a label per each object in uS.

In [8]:
smr

[[78.0, 0.9257, 7.4006],
 [77.0, 0.9259, 6.9821],
 [76.0, 0.9255, 6.9672],
 [75.0, 0.9254, 7.0676],
 [74.0, 0.9254, 6.9307],
 [73.0, 0.9256, 6.916],
 [72.0, 0.9255, 7.0556],
 [71.0, 0.9253, 7.2611],
 [70.0, 0.9258, 6.8333],
 [69.0, 0.9254, 6.9441],
 [68.0, 0.9256, 6.9006],
 [67.0, 0.9255, 7.397],
 [66.0, 0.9254, 7.0745],
 [65.0, 0.9251, 6.9306],
 [64.0, 0.9254, 7.0281],
 [63.0, 0.9255, 7.0172],
 [62.0, 0.9256, 6.782],
 [61.0, 0.9254, 6.7463],
 [60.0, 0.9258, 6.9929],
 [59.0, 0.9254, 6.6483],
 [58.0, 0.9258, 6.6164],
 [57.0, 0.9259, 6.631],
 [56.0, 0.9257, 6.6714],
 [55.0, 0.9258, 6.6261],
 [54.0, 0.9256, 6.7244],
 [53.0, 0.9258, 6.9454],
 [52.0, 0.9254, 6.8396],
 [51.0, 0.9255, 6.6728],
 [50.0, 0.9254, 6.6338],
 [49.0, 0.9256, 6.6403],
 [48.0, 0.9257, 6.5409],
 [47.0, 0.9256, 6.6091],
 [46.0, 0.9257, 6.4734],
 [45.0, 0.9256, 6.4937],
 [44.0, 0.9259, 6.487],
 [43.0, 0.9255, 6.4956],
 [42.0, 0.9258, 6.4518],
 [41.0, 0.9255, 6.653],
 [40.0, 0.9259, 6.6654],
 [39.0, 0.9256, 6.496],
 [38.0,